### Load Dataset with T5Tokenizer

In [1]:
# Download https://github.com/google/sentencepiece#installation


import sys
sys.path.append('/root/dev/ColabProtT5/saprot')

dataset_config = {
    "dataloader_kwargs": {
        "batch_size": 1,
        "num_workers": 2
    },
    "train_lmdb": "/root/dataset/DeepLoc/cls2/foldseek/train",
    "valid_lmdb": "/root/dataset/DeepLoc/cls2/foldseek/valid",
    "test_lmdb": "/root/dataset/DeepLoc/cls2/foldseek/test",
    "tokenizer": "/root/models/prot_t5_xl_uniref50",
    "plddt_threshold": None
}

from dataset.protT5.protT5_classification_dataset import ProtT5ClassificationDataset
data_module = ProtT5ClassificationDataset(**dataset_config)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/anaconda3/envs/SaprotHub/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Load T5 Model


In [2]:

model_config = {
    "kwargs":{
        "config_path": "/root/models/prot_t5_xl_uniref50", 
        "load_pretrained": True,
        "num_labels": 2
    },
    "lr_scheduler_kwargs": {
        "class": "ConstantLRScheduler",
        "init_lr": 1e-4
    }
}

print(model_config)

from model.protT5.protT5_classification_model import ProtT5ClassificationModel
model_module = ProtT5ClassificationModel(**model_config["kwargs"])

/opt/anaconda3/envs/SaprotHub/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


{'kwargs': {'config_path': '/root/models/prot_t5_xl_uniref50', 'load_pretrained': True, 'num_labels': 2}, 'lr_scheduler_kwargs': {'class': 'ConstantLRScheduler', 'init_lr': 0.0001}}


/opt/anaconda3/envs/SaprotHub/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/anaconda3/envs/SaprotHub/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at /root/models/prot_t5_xl_uniref50 and are newly initialized: ['classification_head.dens

No lr_scheduler_kwargs provided. The default learning rate is 0.
No optimizer_kwargs provided. The default optimizer is AdamW.


In [3]:
import pytorch_lightning as pl

trainer_config = {
    "max_epochs": 5,
    "log_every_n_steps": 1,
    # "strategy":{
    #     "class": "auto"
    # },
    "logger": False,
    "enable_checkpointing": False,
    "val_check_interval": 0.25,
    "accelerator": "gpu",
    "devices": 1,
    "num_nodes": 1,
    "accumulate_grad_batches": 1,
    "precision": 16,
    "num_sanity_val_steps": 0
}

trainer = pl.Trainer(**trainer_config, callbacks=[], use_distributed_sampler=False)

/opt/anaconda3/envs/SaprotHub/lib/python3.10/site-packages/lightning_fabric/connector.py:558: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model=model_module, datamodule=data_module)

You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name      | Type                        | Params
----------------------------------------------------------
0 | model     | T5ForSequenceClassification | 2.8 B 
1 | train_acc | Accuracy                    | 0     
2 | valid_acc | Accuracy                    | 0     
3 | test_acc  | Accuracy                    | 0     
----------------------------------------------------------
2.8 B     Trainable params
0         Non-trainable params
2.8 B     Total params
11,279.528Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 44.43 GiB total capacity; 14.28 GiB already allocated; 25.81 MiB free; 14.66 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF